In [ ]:
import tkinter as tk
from tkinter import ttk 
import numpy as np 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt 
from xml.dom import minidom
import csv
import time as tm
import datetime

class Demo(tk.Frame):

	def __init__(self, parent=None):
		tk.Frame.__init__(self, parent)
		self.pack()
		self.GUI()

	def GUI(self):
		self.frameUserIndicator = tk.Canvas(self)
		self.frameControlPanel = tk.Canvas(self)
		self.frameTableTop = tk.Canvas(self)

		self.frameUserIndicator.grid(row=0, column=0)
		self.frameControlPanel.grid(row=0, column=1)
		self.frameTableTop.grid(row=1, column=0, columnspan=2)

		# user indicators
		amountUsers = 2
		self.userIndicators = [0 for i in range(amountUsers)]
		for i in range(amountUsers):
			self.userIndicators[i] = tk.Label(self.frameUserIndicator, 
									text="User "+str(i+1), background='green', foreground='white')
			self.userIndicators[i].grid(row=0, column=i)

		# control frame
		self.btnStart = tk.Button(self.frameControlPanel, text="Start", width=10)
		self.btnStart.grid(row=0, column=0)
		self.btnStart.bind('<Button-1>', self.groupTouch)

		# PixelSense tabletop
		self.tabletop = tk.Canvas(self.frameTableTop, borderwidth=2, relief=tk.GROOVE, width=651, height=419.4)
		self.tabletop.grid(row=0, column=0)

	def groupTouch(event):
		print("groupTouch()")

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Group Touch Demo")
    root.geometry("1000x600")
    root.resizable(width=False, height=False)
    Demo(root).mainloop()